In [1]:
import os
import os.path
import re
import sys
import codecs
import pandas as pd
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='bs4')
import numpy as np # linear algebra
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
#reload(sys)
#sys.setdefaultencoding('utf-8')

In [6]:
path = "C:\\Users\\24596\\3D Objects\\data"
#把e:get_key目录下的文件名全部获取保存在files中
files = os.listdir(path)
s=[]
for file in files:
    content=open(file, encoding="utf-8").read()
    s.append(content)
data1=pd.DataFrame()
data1["text"]=s

In [7]:
lemmatizer = WordNetLemmatizer()
#删除url 统一资源定位符
def rmURL(line):
    import re
    pattern = re.compile(r'http[a-zA-Z0-9.?/&=:]*')
    return pattern.sub('', line.strip())

# 去非中文、英文、数字字符， 如表情，小语种字符
def rmUNK(line):
    import re
    pattern = re.compile(u'[^0-9a-zA-Z\u4e00-\u9fa5.，,。？“”]+', re.UNICODE)
    return pattern.sub('', line.strip())

##### 去除html标签
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

#### 删除标点符号
from string import punctuation
def remove_punc(text):
    dicts={i:'' for i in punctuation}
    punc_table=str.maketrans(dicts)
    new_s=text.translate(punc_table)
    return new_s

### 文本清洗

In [11]:
data1["text_clean"]=data1["text"].apply(lambda x:rmURL(x))
data1["text_clean"]=data1["text_clean"].apply(lambda x:rmUNK(x))
data1["text_clean"]=data1["text_clean"].apply(lambda x:remove_html(x))
#data1['token'] = data1['text'].apply(word_tokenize) 

,text,text_clean
0,我正式开始希望渺茫的调查是狗案案发的第三天以后。\n随着调查的进行我不无遗憾地感到再现历史是...,我正式开始希望渺茫的调查是狗案案发的第三天以后。随着调查的进行我不无遗憾地感到再现历史是件困...
1,12\n方春英来电话催问，能不能安葬索姗。\n＂当然可以。＂\n我道着歉意，＂真对不起，我们...,12方春英来电话催问，能不能安葬索姗。当然可以。我道着歉意，真对不起，我们耽搁了太多的时间。...
2,＂警坛一绝＂张欣\n古城门口，人群熙熙攘攘。\n城墙上高挂着悬赏捉拿的要犯的头像。\n混杂在...,警坛一绝张欣古城门口，人群熙熙攘攘。城墙上高挂着悬赏捉拿的要犯的头像。混杂在人群中的图中人见...
3,设计死亡\n彭祖贻\n夜晚总给人以神秘的感觉。\n关于这天晚上的情景，我告诉刑警吴明说，我到...,设计死亡彭祖贻夜晚总给人以神秘的感觉。关于这天晚上的情景，我告诉刑警吴明说，我到江边去的时候...
4,当公安人员拿着这幅模拟画像发动厂内群众辨认时，群众一下子认出：凶手就是该厂装配车间工人胡茂军...,当公安人员拿着这幅模拟画像发动厂内群众辨认时，群众一下子认出凶手就是该厂装配车间工人胡茂军。...


### jieba分词

In [28]:
def split_word(s):
    seg=jieba.cut(s,cut_all=False)
    return ",".join(seg)
data1["split_text"]=data1["text_clean"].apply(lambda x : split_word(x))

In [35]:
data1

,text,text_clean,split_text
0,我正式开始希望渺茫的调查是狗案案发的第三天以后。\n随着调查的进行我不无遗憾地感到再现历史是...,我正式开始希望渺茫的调查是狗案案发的第三天以后。随着调查的进行我不无遗憾地感到再现历史是件困...,"我,正式,开始,希望,渺茫,的,调查,是,狗案,案发,的,第三天,以后,。,随着,调查,的,..."
1,12\n方春英来电话催问，能不能安葬索姗。\n＂当然可以。＂\n我道着歉意，＂真对不起，我们...,12方春英来电话催问，能不能安葬索姗。当然可以。我道着歉意，真对不起，我们耽搁了太多的时间。...,"12,方春英来,电话,催问,，,能,不能,安葬,索姗,。,当然,可以,。,我道,着,歉意,，..."
2,＂警坛一绝＂张欣\n古城门口，人群熙熙攘攘。\n城墙上高挂着悬赏捉拿的要犯的头像。\n混杂在...,警坛一绝张欣古城门口，人群熙熙攘攘。城墙上高挂着悬赏捉拿的要犯的头像。混杂在人群中的图中人见...,"警坛,一绝,张欣,古城,门口,，,人群,熙熙攘攘,。,城墙,上,高,挂,着,悬赏,捉拿,的,..."
3,设计死亡\n彭祖贻\n夜晚总给人以神秘的感觉。\n关于这天晚上的情景，我告诉刑警吴明说，我到...,设计死亡彭祖贻夜晚总给人以神秘的感觉。关于这天晚上的情景，我告诉刑警吴明说，我到江边去的时候...,"设计,死亡,彭祖,贻,夜晚,总,给人以,神秘,的,感觉,。,关于,这天,晚上,的,情景,，,..."
4,当公安人员拿着这幅模拟画像发动厂内群众辨认时，群众一下子认出：凶手就是该厂装配车间工人胡茂军...,当公安人员拿着这幅模拟画像发动厂内群众辨认时，群众一下子认出凶手就是该厂装配车间工人胡茂军。...,"当,公安人员,拿,着,这幅,模拟,画像,发动,厂内,群众,辨认,时,，,群众,一下子,认出,..."
5,不是冤家不聚头\n曾纪鑫\n1、夜半谋杀\n每学期一次的教学检查正在古船县实验小学如期举行。...,不是冤家不聚头曾纪鑫1夜半谋杀每学期一次的教学检查正在古船县实验小学如期举行。按计划安排，三...,"不是冤家不聚头,曾纪鑫,1,夜半,谋杀,每学期,一次,的,教学检查,正在,古船,县,实验,小..."
6,三、可怕的梦魇\n四人立即禁声。\n心道：这么晚了，会是谁呢？\n难道也是夜行迷路之人？\n...,三可怕的梦魇四人立即禁声。心道这么晚了，会是谁呢？难道也是夜行迷路之人？四人屏住呼吸，隐身暗...,"三,可怕,的,梦魇,四人,立即,禁声,。,心道,这么晚,了,，,会,是,谁,呢,？,难道,也..."
7,八、致命的侥幸心理\n杨国柱已经好几晚睡不踏实了。\n这是什么地方？\n警察对怀疑有问题的人...,八致命的侥幸心理杨国柱已经好几晚睡不踏实了。这是什么地方？警察对怀疑有问题的人搓澡的地方。不...,"八,致命,的,侥幸心理,杨国柱,已经,好几,晚睡,不,踏实,了,。,这是,什么,地方,？,警..."
8,２６．黑蔷薇案件到这时才刚刚揭开序幕，好在重场戏不怕晚，公安部全力支持，调直升飞机送金剑启程...,黑蔷薇案件到这时才刚刚揭开序幕，好在重场戏不怕晚，公安部全力支持，调直升飞机送金剑启程金剑带...,"黑,蔷薇,案件,到,这时,才,刚刚,揭开序幕,，,好,在,重场,戏,不怕,晚,，,公安部,全..."
9,法医张维贤的验尸生涯\n上海市长宁区公安分局治安科长兼法医张维贤，在２７年法医生涯中，验过１...,法医张维贤的验尸生涯上海市长宁区公安分局治安科长兼法医张维贤，在年法医生涯中，验过具死因不明...,"法医,张维贤,的,验尸,生涯,上海市,长宁区,公安分局,治安,科长,兼,法医,张维贤,，,在..."
